<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/improved_viratt_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

improved Viratt Strategy

In [1]:
pip install ccxt pandas numpy ta scikit-learn matplotlib seaborn requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 9.6 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=013ac77f14a81a435fec3d513f2d6b434ff29e1c6f363b05718391913aaa49d5
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [6]:
import ccxt
import pandas as pd
import numpy as np

def fetch_signals_optimized(api, assets):
    """
    Fetches real-time trading signals based on OBV, RSI, and ATR.

    Args:
        api: CCXT exchange instance.
        assets: List of asset pairs to evaluate.

    Returns:
        List of signals for each asset.
    """
    results = []

    for asset in assets:
        try:
            # Fetch data from KuCoin
            ohlcv = api.fetch_ohlcv(asset, timeframe="1h", limit=100)
            df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])

            # Convert timestamp to datetime for better readability
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

            # Calculate OBV
            df['obv'] = (np.sign(df['close'].diff()) * df['volume']).cumsum()

            # Calculate RSI
            delta = df['close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            df['rsi'] = 100 - (100 / (1 + rs))

            # Calculate ATR
            df['tr'] = np.maximum(df['high'] - df['low'],
                                  np.maximum(abs(df['high'] - df['close'].shift()),
                                             abs(df['low'] - df['close'].shift())))
            df['atr'] = df['tr'].rolling(window=14).mean()

            # Signal generation logic
            df['obv_change'] = df['obv'].pct_change()
            df['signal'] = np.where(
                (df['obv_change'] > 0.01) & (df['rsi'] > 50) & (df['atr'] > 0.005),
                "Buy",
                np.where((df['obv_change'] < -0.01) & (df['rsi'] < 50) & (df['atr'] > 0.005),
                         "Sell",
                         "Hold")
            )

            latest = df.iloc[-1]  # Get the latest row
            results.append({
                "asset": asset,
                "signal": latest['signal'],
                "obv_change": latest['obv_change'],
                "rsi": latest['rsi'],
                "atr": latest['atr'],
                "close_price": latest['close']
            })
        except Exception as e:
            print(f"Error processing {asset}: {e}")

    return results

def display_signals(signals):
    """
    Displays the signals in a detailed and user-friendly format.

    Args:
        signals: List of signals with asset details.
    """
    print("\n--- Trading Signals ---")
    for signal in signals:
        print(f"Asset: {signal['asset']}")
        print(f"   Signal: {signal['signal']}")
        print(f"   Current Price: ${signal['close_price']:.2f}")
        print(f"   OBV Change: {signal['obv_change']:.4f} (Positive = Buying Pressure; Negative = Selling Pressure)")
        print(f"   RSI: {signal['rsi']:.2f} (Above 50: Bullish; Below 50: Bearish)")
        print(f"   ATR: {signal['atr']:.5f} (Higher indicates more volatility)")
        print()

# Initialize CCXT for KuCoin
exchange = ccxt.kucoin()

# Define the list of assets
assets = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "NEAR/USDT",
          "XRP/USDT", "AVAX/USDT", "LINK/USDT", "CFX/USDT",
          "XAVA/USDT", "STG/USDT" , "FET/USDT"]

# Fetch signals
signals = fetch_signals_optimized(exchange, assets)

# Display results
display_signals(signals)



--- Trading Signals ---
Asset: BTC/USDT
   Signal: Sell
   Current Price: $104752.30
   OBV Change: -0.0805 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 48.24 (Above 50: Bullish; Below 50: Bearish)
   ATR: 397.73571 (Higher indicates more volatility)

Asset: ETH/USDT
   Signal: Sell
   Current Price: $3305.56
   OBV Change: -0.1280 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 48.45 (Above 50: Bullish; Below 50: Bearish)
   ATR: 20.75429 (Higher indicates more volatility)

Asset: ADA/USDT
   Signal: Buy
   Current Price: $0.98
   OBV Change: 0.0906 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 57.67 (Above 50: Bullish; Below 50: Bearish)
   ATR: 0.00821 (Higher indicates more volatility)

Asset: NEAR/USDT
   Signal: Sell
   Current Price: $4.93
   OBV Change: -0.1035 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 40.94 (Above 50: Bullish; Below 50: Bearish)
   ATR: 0.05049 (Higher indicates more volatility

In [4]:
import ccxt
import pandas as pd
import numpy as np

def fetch_signals_and_backtest(api, assets):
    """
    Fetches trading signals based on OBV, RSI, and ATR and backtests the strategy.

    Args:
        api: CCXT exchange instance.
        assets: List of asset pairs to evaluate.

    Returns:
        List of results with trades, gains, and signals for each asset.
    """
    results = []
    total_trades = 0
    total_returns = 0

    for asset in assets:
        try:
            # Fetch data from KuCoin
            ohlcv = api.fetch_ohlcv(asset, timeframe="1h", limit=720)  # Approx. 1 month of data
            df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])

            # Convert timestamp to datetime for better readability
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

            # Calculate OBV
            df['obv'] = (np.sign(df['close'].diff()) * df['volume']).cumsum()

            # Calculate RSI
            delta = df['close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            df['rsi'] = 100 - (100 / (1 + rs))

            # Calculate ATR
            df['tr'] = np.maximum(df['high'] - df['low'],
                                  np.maximum(abs(df['high'] - df['close'].shift()),
                                             abs(df['low'] - df['close'].shift())))
            df['atr'] = df['tr'].rolling(window=14).mean()

            # Signal generation logic
            df['obv_change'] = df['obv'].pct_change()
            df['signal'] = np.where(
                (df['obv_change'] > 0.01) & (df['rsi'] > 50) & (df['atr'] > 0.005),
                "Buy",
                np.where((df['obv_change'] < -0.01) & (df['rsi'] < 50) & (df['atr'] > 0.005),
                         "Sell",
                         "Hold")
            )

            # Backtest logic
            initial_capital = 1000
            position = 0
            capital = initial_capital
            num_trades = 0
            buy_price = 0

            for i, row in df.iterrows():
                if row['signal'] == "Buy" and position == 0:  # Buy
                    position = capital / row['close']
                    buy_price = row['close']
                    capital = 0
                    num_trades += 1
                elif row['signal'] == "Sell" and position > 0:  # Sell
                    capital = position * row['close']
                    position = 0
                    num_trades += 1

            # Final position liquidation
            if position > 0:
                capital = position * df.iloc[-1]['close']
                position = 0

            total_gains = capital - initial_capital
            total_trades += num_trades
            total_returns += total_gains

            latest = df.iloc[-1]
            results.append({
                "asset": asset,
                "signal": latest['signal'],
                "obv_change": latest['obv_change'],
                "rsi": latest['rsi'],
                "atr": latest['atr'],
                "close_price": latest['close'],
                "num_trades": num_trades,
                "gains": total_gains
            })
        except Exception as e:
            print(f"Error processing {asset}: {e}")

    return results, total_trades, total_returns

def display_results(results, total_trades, total_returns):
    """
    Displays the results of backtesting in a detailed and user-friendly format.

    Args:
        results: List of results with trades, gains, and signals.
        total_trades: Total number of trades across all assets.
        total_returns: Total returns across all assets.
    """
    print("\n--- Trading Results ---")
    for result in results:
        print(f"Asset: {result['asset']}")
        print(f"   Signal: {result['signal']}")
        print(f"   Current Price: ${result['close_price']:.2f}")
        print(f"   OBV Change: {result['obv_change']:.4f} (Positive = Buying Pressure; Negative = Selling Pressure)")
        print(f"   RSI: {result['rsi']:.2f} (Above 50: Bullish; Below 50: Bearish)")
        print(f"   ATR: {result['atr']:.5f} (Higher indicates more volatility)")
        print(f"   Number of Trades: {result['num_trades']}")
        print(f"   Gains: ${result['gains']:.2f}")
        print()

    print("\n--- Overall Performance ---")
    print(f"Total Trades: {total_trades}")
    print(f"Total Returns: ${total_returns:.2f}")

# Initialize CCXT for KuCoin
exchange = ccxt.kucoin()

# Define the list of assets
assets = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "NEAR/USDT",
          "XRP/USDT", "AVAX/USDT", "LINK/USDT", "CFX/USDT",
          "XAVA/USDT", "STG/USDT", "FET/USDT"]

# Fetch signals and backtest
results, total_trades, total_returns = fetch_signals_and_backtest(exchange, assets)

# Display results
display_results(results, total_trades, total_returns)



--- Trading Results ---
Asset: BTC/USDT
   Signal: Hold
   Current Price: $104633.60
   OBV Change: 0.0089 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 47.95 (Above 50: Bullish; Below 50: Bearish)
   ATR: 410.92143 (Higher indicates more volatility)
   Number of Trades: 54
   Gains: $-53.53

Asset: ETH/USDT
   Signal: Sell
   Current Price: $3304.95
   OBV Change: -0.0636 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 43.43 (Above 50: Bullish; Below 50: Bearish)
   ATR: 21.69000 (Higher indicates more volatility)
   Number of Trades: 62
   Gains: $-107.96

Asset: ADA/USDT
   Signal: Hold
   Current Price: $0.98
   OBV Change: 0.0033 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 56.37 (Above 50: Bullish; Below 50: Bearish)
   ATR: 0.00830 (Higher indicates more volatility)
   Number of Trades: 56
   Gains: $20.33

Asset: NEAR/USDT
   Signal: Hold
   Current Price: $4.92
   OBV Change: 0.0137 (Positive = Buying Pressure; Negati

In [5]:
import ccxt
import pandas as pd
import numpy as np

def fetch_signals_and_backtest(api, assets):
    """
    Fetches trading signals based on OBV, RSI, and ATR and backtests the strategy.

    Args:
        api: CCXT exchange instance.
        assets: List of asset pairs to evaluate.

    Returns:
        List of results with trades, gains, success ratio, and signals for each asset.
    """
    results = []
    total_trades = 0
    total_returns = 0
    total_successful_trades = 0
    total_profitable_trades = 0  # Tracks total successful trades for all assets

    for asset in assets:
        try:
            # Fetch data from KuCoin
            ohlcv = api.fetch_ohlcv(asset, timeframe="1h", limit=720)  # Approx. 1 month of data
            df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])

            # Convert timestamp to datetime for better readability
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

            # Calculate OBV
            df['obv'] = (np.sign(df['close'].diff()) * df['volume']).cumsum()

            # Calculate RSI
            delta = df['close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            df['rsi'] = 100 - (100 / (1 + rs))

            # Calculate ATR
            df['tr'] = np.maximum(df['high'] - df['low'],
                                  np.maximum(abs(df['high'] - df['close'].shift()),
                                             abs(df['low'] - df['close'].shift())))
            df['atr'] = df['tr'].rolling(window=14).mean()

            # Signal generation logic
            df['obv_change'] = df['obv'].pct_change()
            df['signal'] = np.where(
                (df['obv_change'] > 0.01) & (df['rsi'] > 50) & (df['atr'] > 0.005),
                "Buy",
                np.where((df['obv_change'] < -0.01) & (df['rsi'] < 50) & (df['atr'] > 0.005),
                         "Sell",
                         "Hold")
            )

            # Backtest logic
            initial_capital = 1000
            position = 0
            capital = initial_capital
            num_trades = 0
            successful_trades = 0
            buy_price = 0
            profitable_trades = 0  # Tracks successful trades for this asset

            for i, row in df.iterrows():
                if row['signal'] == "Buy" and position == 0:  # Buy
                    position = capital / row['close']
                    buy_price = row['close']
                    capital = 0
                    num_trades += 1
                elif row['signal'] == "Sell" and position > 0:  # Sell
                    sell_price = row['close']
                    capital = position * sell_price
                    position = 0
                    num_trades += 1
                    # Check if trade was profitable
                    if sell_price > buy_price:
                        profitable_trades += 1

            # Final position liquidation
            if position > 0:
                capital = position * df.iloc[-1]['close']
                position = 0

            total_gains = capital - initial_capital
            success_ratio = (profitable_trades / (num_trades // 2) * 100) if num_trades > 0 else 0
            total_trades += num_trades
            total_returns += total_gains
            total_profitable_trades += profitable_trades

            latest = df.iloc[-1]
            results.append({
                "asset": asset,
                "signal": latest['signal'],
                "obv_change": latest['obv_change'],
                "rsi": latest['rsi'],
                "atr": latest['atr'],
                "close_price": latest['close'],
                "num_trades": num_trades,
                "gains": total_gains,
                "success_ratio": success_ratio
            })
        except Exception as e:
            print(f"Error processing {asset}: {e}")

    # Calculate total success ratio
    total_success_ratio = (total_profitable_trades / (total_trades // 2) * 100) if total_trades > 0 else 0
    return results, total_trades, total_returns, total_success_ratio

def display_results(results, total_trades, total_returns, total_success_ratio):
    """
    Displays the results of backtesting in a detailed and user-friendly format.

    Args:
        results: List of results with trades, gains, success ratio, and signals.
        total_trades: Total number of trades across all assets.
        total_returns: Total returns across all assets.
        total_success_ratio: Overall success ratio across all assets.
    """
    print("\n--- Trading Results ---")
    for result in results:
        print(f"Asset: {result['asset']}")
        print(f"   Signal: {result['signal']}")
        print(f"   Current Price: ${result['close_price']:.2f}")
        print(f"   OBV Change: {result['obv_change']:.4f} (Positive = Buying Pressure; Negative = Selling Pressure)")
        print(f"   RSI: {result['rsi']:.2f} (Above 50: Bullish; Below 50: Bearish)")
        print(f"   ATR: {result['atr']:.5f} (Higher indicates more volatility)")
        print(f"   Number of Trades: {result['num_trades']}")
        print(f"   Gains: ${result['gains']:.2f}")
        print(f"   Success Ratio: {result['success_ratio']:.2f}%")
        print()

    print("\n--- Overall Performance ---")
    print(f"Total Trades: {total_trades}")
    print(f"Total Returns: ${total_returns:.2f}")
    print(f"Overall Success Ratio: {total_success_ratio:.2f}%")

# Initialize CCXT for KuCoin
exchange = ccxt.kucoin()

# Define the list of assets
assets = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "NEAR/USDT",
          "XRP/USDT", "AVAX/USDT", "LINK/USDT", "CFX/USDT",
          "XAVA/USDT", "STG/USDT", "FET/USDT"]

# Fetch signals and backtest
results, total_trades, total_returns, total_success_ratio = fetch_signals_and_backtest(exchange, assets)

# Display results
display_results(results, total_trades, total_returns, total_success_ratio)


Error processing CFX/USDT: division by zero

--- Trading Results ---
Asset: BTC/USDT
   Signal: Sell
   Current Price: $104754.70
   OBV Change: -0.0211 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 48.29 (Above 50: Bullish; Below 50: Bearish)
   ATR: 397.73571 (Higher indicates more volatility)
   Number of Trades: 56
   Gains: $-46.45
   Success Ratio: 32.14%

Asset: ETH/USDT
   Signal: Sell
   Current Price: $3302.04
   OBV Change: -0.1368 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 47.27 (Above 50: Bullish; Below 50: Bearish)
   ATR: 20.75429 (Higher indicates more volatility)
   Number of Trades: 62
   Gains: $-102.97
   Success Ratio: 35.48%

Asset: ADA/USDT
   Signal: Hold
   Current Price: $0.98
   OBV Change: -0.0065 (Positive = Buying Pressure; Negative = Selling Pressure)
   RSI: 58.62 (Above 50: Bullish; Below 50: Bearish)
   ATR: 0.00821 (Higher indicates more volatility)
   Number of Trades: 58
   Gains: $-22.30
   Success Ratio: 